## 종목별 수익률 미분계수 구하기
### 심볼릭 연산

https://datascienceschool.net/view-notebook/5f5f4a966f9042efa43d711008df9c0b/

In [2]:
import FinanceDataReader as fdr
fdr.__version__

'0.6.0'

In [3]:
# import FinanceDataReader as fdr

# 한국거래소 상장종목 전체
df_krx = fdr.StockListing('KRX')
df_krx.head()

,Symbol,Name,Sector,Industry
0,001040,CJ,기타 금융업,지주회사
1,011150,CJ씨푸드,기타 식품 제조업,"수산물(어묵,맛살)가공품 도매,원양수산업,수출입"
2,012630,HDC,건물 건설업,"토목공사,건축공사,아파트분양사업,재개발/재건축사업"
3,082740,HSD엔진,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진"
4,001390,KG케미칼,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제"


In [4]:
import numpy as np 
import pandas as pd 

In [5]:
def stock_select(code,strt_year='2012', end_date='2018-09-30'):

    stock_df = fdr.DataReader(code,strt_year,end_date)

    # Date를 인덱스에서 제거하고 항목으로 사용
    stock_df = stock_df.reset_index()

    # Date에서 연도를 추출함
    stock_df['Year'] = stock_df.Date.map(lambda x : x.year )
    stock_df['Code'] = code

    return stock_df


## 다항식 만들기

In [22]:
import numpy as np

def updown_poly(up_days,down_days):
    
    # 3차 다항식생성
    x = list(range(1,len(up_days)+1))
    up_p3 = np.polyfit(x,up_days,3)
    x = list(range(1,len(down_days)+1))
    down_p3 = np.polyfit(x,down_days,3)
    return up_p3, down_p3



In [32]:
# 출처: http://codepractice.tistory.com/76 [코딩 연습]
# Juypter 노트북에서 수학식의 LaTeX 표현을 위해 필요함
# sympy.init_printing(use_latex='mathjax')
from sympy import symbols, Limit
from sympy import Derivative

def updown_strength(up_days, down_days):
    
    # 함수 생성
    up_m, down_m = updown_poly(up_days, down_days)

    x, a, h = symbols('x, a, h')

    # 함수 정의
    up_fx = up_m[0] * x ** 3 + up_m[1] * x ** 2 + up_m[2] * x + up_m[3]
    down_fx = down_m[0] * x ** 3 + down_m[1] * x ** 2 + down_m[2] * x + down_m[3]


    # up_미분 -------------------------

    # 1차 도함수
    up_d1 = Derivative(up_fx, x).doit()
    # 2차 도함수
    up_d2 = Derivative(up_fx, x, 2).doit()

    # 미분계수
    up_d1_coef = []
    up_d2_coef = []
    for value in range(1,len(up_days)+1):        
        up_d1_coef = up_d1_coef.append(up_d1.subs({x:value}) # x 숫자 지정
        up_d2_coef = up_d2_coef.append(up_d2.subs({x:value}) # x 숫자 지정


    print("up함수 : ", up_d1_coef,up_d2_coef)

    # down_미분 -------------------------

    # 1차 도함수
    down_d1 = Derivative(down_fx, x).doit()
    # 2차 도함수
    down_d2 = Derivative(down_fx, x, 2).doit()

    # 미분계수
    down_d1_coef = []
    down_d2_coef = []
    for value in range(1,len(down_days)+1):        
        down_d1_coef = down_d1_coef.append(down_d1.subs({x:value}) # x 숫자 지정
        down_d2_coef = down_d2_coef.append(down_d2.subs({x:value}) # x 숫자 지정


    print("down함수 : ", down_d1_coef,down_d2_coef)
    
    return up_d1_coef, up_d2_coef, down_d1_coef, down_d2_coef

SyntaxError: invalid syntax (<ipython-input-32-d43b8bcdd878>, line 31)

In [29]:
stock_df = stock_select('005930')
v_d1_coef, v_d2_coef, c_d1_coef, c_d2_coef = updown_strength(stock_df.Volume, stock_df.Change)
print(v_d1_coef, v_d2_coef, c_d1_coef, c_d2_coef)

up함수 :  -21906.2801842085 63.4674709885734
down함수 :  -0.0100112778691156 4.48194518146954e-5
-21906.2801842085 63.4674709885734 -0.0100112778691156 4.48194518146954e-5


In [ ]:
print()